In [3]:
!pip install google-play-scraper
!pip install konlpy
!pip uninstall JPype1-py3
!pip install JPype1


In [7]:
import re
import json
from google_play_scraper import reviews, Sort
from konlpy.tag import Okt
from collections import Counter

def fetch_reviews(app_id, total_reviews=1000, max_attempts=30):
    """Google Play Store에서 리뷰를 수집합니다."""
    all_reviews = []
    count = 100
    token = None
    attempts = 0
    previous_token = None

    while len(all_reviews) < total_reviews and attempts < max_attempts:
        print(f"Fetching... ({len(all_reviews)} / {total_reviews})")
        result, token = reviews(
            app_id,
            lang='ko',
            country='kr',
            sort=Sort.NEWEST,
            count=count,
            continuation_token=token
        )

        # 무한 반복 방지
        if token == previous_token:
            print("🔁 같은 continuation token 반복 감지됨. 종료.")
            break
        previous_token = token

        all_reviews.extend(result)

        if token is None:
            break
        attempts += 1

    # 중복 제거
    unique_reviews = [dict(t) for t in {tuple(d.items()) for d in all_reviews}]
    print(f"✅ 수집 완료: {len(unique_reviews)}개 (중복 제거됨)")
    return unique_reviews[:total_reviews]

def analyze_reviews(review_list):
    """명사를 추출하고 상위 키워드를 분석합니다."""
    okt = Okt()
    all_text = " ".join([review['content'] for review in review_list])
    all_text = re.sub(r'[^가-힣\s]', '', all_text)
    nouns = okt.nouns(all_text)

    stopwords = set([
        '이', '그', '저', '것', '들', '의', '있', '하', '되', '수',
        '때문', '절대', '원래', '관련', '아예', '읍니', '만하', '에드', '리기'
    ])
    nouns = [noun for noun in nouns if noun not in stopwords and len(noun) > 1]
    counter = Counter(nouns)
    return counter.most_common(10)

def group_reviews_by_rating(review_list, target_count=2000):
    """평점별로 최대 N개의 리뷰를 그룹화합니다."""
    grouped = {1: [], 2: [], 3: [], 4: [], 5: []}
    for review in review_list:
        score = review.get('score')
        if score in grouped and len(grouped[score]) < target_count:
            grouped[score].append(review)
    return grouped

if __name__ == "__main__":
    app_id = 'com.toodat.android'  # 모픽 앱

    print("📥 리뷰 수집 중...")
    reviews_data = fetch_reviews(app_id, total_reviews=2000)
    total_reviews = len(reviews_data)

    overall_counts = Counter(review['score'] for review in reviews_data)

    print("\n📊 평점별 리뷰 개수:")
    for score in range(1, 6):
        print(f"{score}점 리뷰: {overall_counts.get(score, 0)}개")

    grouped_reviews = group_reviews_by_rating(reviews_data, target_count=2000)

    analysis_result = {}
    for score in sorted(grouped_reviews.keys()):
        reviews_for_score = grouped_reviews[score]
        print(f"\n⭐ 평점 {score}점 리뷰 {len(reviews_for_score)}개 분석 중...")
        if reviews_for_score:
            top_words = analyze_reviews(reviews_for_score)
            for rank, (word, freq) in enumerate(top_words, start=1):
                print(f"{rank}위: {word} - {freq}회")
            analysis_result[str(score)] = {
                "review_count": len(reviews_for_score),
                "top_words": [{"word": word, "frequency": freq} for word, freq in top_words]
            }
        else:
            print("리뷰 없음.")
            analysis_result[str(score)] = {
                "review_count": 0,
                "top_words": []
            }

    result_data = {
        "total_reviews": total_reviews,
        "overall_counts": {str(score): overall_counts.get(score, 0) for score in range(1, 6)},
        "analysis": analysis_result
    }

    with open("analysis_result.json", "w", encoding="utf-8") as f:
        json.dump(result_data, f, ensure_ascii=False, indent=2)

    print("\n✅ 분석 결과가 'analysis_result.json'에 저장되었습니다.")

📥 리뷰 수집 중...
Fetching... (0 / 2000)
Fetching... (34 / 2000)
🔁 같은 continuation token 반복 감지됨. 종료.
✅ 수집 완료: 34개 (중복 제거됨)

📊 평점별 리뷰 개수:
1점 리뷰: 6개
2점 리뷰: 1개
3점 리뷰: 4개
4점 리뷰: 4개
5점 리뷰: 19개

⭐ 평점 1점 리뷰 6개 분석 중...
1위: 로그인 - 3회
2위: 오류 - 1회
3위: 설치 - 1회
4위: 강요 - 1회
5위: 에러 - 1회
6위: 발생 - 1회
7위: 라며 - 1회
8위: 몇번 - 1회
9위: 시도 - 1회
10위: 인생 - 1회

⭐ 평점 2점 리뷰 1개 분석 중...
1위: 버그 - 2회
2위: 다음 - 2회
3위: 장점 - 1회
4위: 피해망상 - 1회
5위: 연애 - 1회
6위: 단점 - 1회
7위: 지금 - 1회
8위: 화로 - 1회
9위: 마지막 - 1회
10위: 소설 - 1회

⭐ 평점 3점 리뷰 4개 분석 중...
1위: 소설 - 3회
2위: 진행 - 1회
3위: 크롤 - 1회
4위: 글자 - 1회
5위: 보기 - 1회
6위: 덜덜 - 1회
7위: 피아 - 1회
8위: 시리즈 - 1회
9위: 반면 - 1회
10위: 정말 - 1회

⭐ 평점 4점 리뷰 4개 분석 중...
1위: 가입 - 4회
2위: 무료 - 2회
3위: 다음 - 2회
4위: 화로 - 2회
5위: 기능 - 2회
6위: 계정 - 2회
7위: 회원 - 2회
8위: 웹소설 - 1회
9위: 크롤 - 1회
10위: 모드 - 1회

⭐ 평점 5점 리뷰 19개 분석 중...
1위: 최고 - 4회
2위: 작품 - 4회
3위: 무료 - 3회
4위: 어플 - 3회
5위: 사용 - 3회
6위: 페이지 - 3회
7위: 기능 - 3회
8위: 다음 - 2회
9위: 다른 - 2회
10위: 소설 - 2회

✅ 분석 결과가 'analysis_result.json'에 저장되었습니다.


In [8]:
print(json.dumps(result_data, ensure_ascii=False, indent=2))

{
  "total_reviews": 34,
  "overall_counts": {
    "1": 6,
    "2": 1,
    "3": 4,
    "4": 4,
    "5": 19
  },
  "analysis": {
    "1": {
      "review_count": 6,
      "top_words": [
        {
          "word": "로그인",
          "frequency": 3
        },
        {
          "word": "오류",
          "frequency": 1
        },
        {
          "word": "설치",
          "frequency": 1
        },
        {
          "word": "강요",
          "frequency": 1
        },
        {
          "word": "에러",
          "frequency": 1
        },
        {
          "word": "발생",
          "frequency": 1
        },
        {
          "word": "라며",
          "frequency": 1
        },
        {
          "word": "몇번",
          "frequency": 1
        },
        {
          "word": "시도",
          "frequency": 1
        },
        {
          "word": "인생",
          "frequency": 1
        }
      ]
    },
    "2": {
      "review_count": 1,
      "top_words": [
        {
          "word": "버그",
         